In [1]:
import sys, os
from threading import Event, Thread
from pybox2d.framework import Framework,Testbed
from pybox2d import *
import pybox2d
import random
import ipywidgets


from pybox2d.framework import Framework,Testbed
from pybox2d import (circle_shape, edge_shape, fixture_def, polygon_shape)

In [2]:
def create_bridge(world, ground, size, offset, plank_count, friction=0.6, density=1.0):
    """
    Create a bridge with plank_count planks,
    utilizing rectangular planks of size (width, height).
    The bridge should start at x_offset, and continue to
    roughly x_offset+width*plank_count.
    The y will not change.
    """
    width, height = size
    x_offset, y_offset = offset
    half_height = height / 2
    plank = fixture_def(
        shape=polygon_shape(box=(width / 2, height / 2)),
        friction=friction,
        density=density,
    )

    bodies = []
    prevBody = ground
    for i in range(plank_count):
        body = world.create_dynamic_body(
            position=(x_offset + width * i, y_offset),
            fixtures=plank,
        )
        bodies.append(body)

        world.create_revolute_joint(
            body_a=prevBody,
            body_b=body,
            anchor=(x_offset + width * (i - 0.5), y_offset)
        )

        prevBody = body

    world.create_revolute_joint(
        body_a=prevBody,
        body_b=ground,
        anchor=(x_offset + width * (plank_count - 0.5), y_offset),
    )
    return bodies


class Bridge (Framework):
    name = "Bridge"
    numPlanks = 30  # Number of planks in the bridge

    def __init__(self, gui):
        super(Bridge, self).__init__(gui)

        # The ground
        ground = self.world.create_body(
            shapes=edge_shape(vertices=[(-40, 0), (40, 0)])
        )

        create_bridge(self.world, ground, (1.0, 0.25),
                      (-14.5, 5), self.numPlanks, 0.2, 20)

        fixture = fixture_def(
            shape=polygon_shape(vertices=[(-0.5, 0.0),
                                           (0.5, 0.0),
                                           (0.0, 1.5),
                                           ]),
            density=1.0
        )
        for i in range(2):
            self.world.create_dynamic_body(
                position=(-8 + 8 * i, 12),
                fixtures=fixture,
            )

        fixture = fixture_def(shape=circle_shape(radius=0.5), density=1)
        for i in range(3):
            self.world.create_dynamic_body(
                position=(-6 + 6 * i, 10),
                fixtures=fixture,
            )

In [3]:
import matplotlib.pyplot as plt

def isclose(a, b, rel_tol=1e-09, abs_tol=0.0):
    return abs(a-b) <= max(rel_tol * max(abs(a), abs(b)), abs_tol)

def scale_color(color):
    return [float(c)/255.0 for c in color]

def rgb(color):
    r = int(color[0])
    g = int(color[1])
    b = int(color[2])
    return f"rgb({r},{g},{b})"

class IPyCanvasDebugDraw(pybox2d.DebugDraw):
    def __init__(self, canvas):
        self.canvas = canvas
        super(IPyCanvasDebugDraw, self).__init__(float_colors=False)

        # self._scale = 30.0
        #self.ppm = ppm
        #self.ippm = 1.0 / self.ppm
        self.outline_width = 0.01
        self.segment_width = 0.01
        self._bounding_box = [ [0,0],[0,0]]

        self.joint_colors = {
            JointType.unknown_joint :   (230, 25, 75),
            JointType.revolute_joint :  (255,0, 0),
            JointType.prismatic_joint : (255, 225, 25),
            JointType.distance_joint :  (0, 130, 200),
            JointType.pulley_joint :    (245, 130, 48),
            JointType.mouse_joint :     (145, 30, 180),
            JointType.gear_joint :      (70, 240, 240),
            JointType.wheel_joint :     (240, 50, 230),
            JointType.weld_joint :      (210, 245, 60),
            JointType.friction_joint :  (250, 190, 190),
            # JointType.rope_joint :      (0, 128, 128),
            JointType.motor_joint :     (230, 190, 255),
        }

    def reset_bounding_box(self):
        self._bounding_box = [ [0,0],[0,0]]

    def _update_bounding_box(self, p):
        for c in range(2):
            if p[c] < self._bounding_box[0][c]:
                self._bounding_box[0][c] = p[c]

            if p[c] > self._bounding_box[1][c]:
                self._bounding_box[1][c] = p[c]

    def draw_solid_circle(self, center, radius, axis, color):
        self.canvas.fill_style = rgb(color)
        self.canvas.fill_circle(center[0], center[1], radius)
        pass
                
    def draw_circle(self, center, radius, color):
        self.canvas.stroke_style = rgb(color)
        self.canvas.stroke_circle(center[0], center[1], radius)

    def draw_point(self, center, size, color):
        radius = 1#self.scale(size/2.0)
        self.canvas.fill_style = rgb(color)
        self.canvas.fill_circle(center[0], center[1], radius)
        
    def draw_segment(self,v1, v2, color):
        self.canvas.stroke_style = rgb(color)
        self.canvas.stroke_line(
            v1[0], v1[1],v2[0], v2[1])
        
            
    def draw_polygon(self,vertices, color):
        self.canvas.stroke_style = rgb(color)
        self.canvas.stroke_polygon(vertices)
        
    def draw_solid_polygon(self,vertices, color):
        self.canvas.fill_style = rgb(color)
        self.canvas.fill_polygon(vertices)

    def draw_particles(self, centers, radius, colors=None):
        pass
    # def draw_joint(self, joint):
    #     # print("draw_joint")
    #     pass
    
    def draw_transform(self, xf):
        pass



import IPython
import time
from ipycanvas import Canvas,MultiCanvas, hold_canvas


class IPyCanvasTestbedGui(object):
    def __init__(self, testbed):
        
        

        self.multi_canvas = MultiCanvas(n_canvases = 2, width=1000, height=1000)
        self.out = ipywidgets.Output()
        self.flip_bit = False

        self.testbed = testbed
        self.framework = self.testbed.exampleCls(gui=self)
        self.framework.gui = self
        self.debug_draw = IPyCanvasDebugDraw(canvas=self.multi_canvas[self.flip_bit])
        print("scale",self.debug_draw.scale)
        self.debug_draw.flip_y = True
        self.debug_draw.translate = (500, 500)
        self.debug_draw.scale = 30
        flags = ['shape','joint','aabb','pair','center_of_mass','particle']
        self.debug_draw.clear_flags(flags)
        flags = ['shape','joint']
        for flag in flags:
            self.debug_draw.append_flags(flag)
        self.world = self.framework.world
        self.world.set_debug_draw(self.debug_draw)
        

        def on_mouse_down( xpos, ypos):
            with self.out:
                print(xpos,ypos)
            self.multi_canvas[1].fill_circle(xpos, ypos, 10)
            self.multi_canvas[1].flush()
            pos = self.debug_draw.screen_to_world((xpos, ypos))
            pos = pos.x, pos.y
            self.framework.on_mouse_down(pos)
            fixture = fixture_def(shape=circle_shape(radius=0.5), density=1)
            # for i in range(3):
            #     self.world.create_dynamic_body(
            #         position=pos,
            #         fixtures=fixture,
            #     )

        def on_mouse_up( xpos, ypos):
            pos = self.debug_draw.screen_to_world((xpos, ypos))
            pos = pos.x, pos.y
            self.framework.on_mouse_up(pos)


        def on_mouse_move( xpos, ypos):
            # self.multi_canvas[1].fill_circle(xpos, ypos, 10)
            # self.multi_canvas[1].flush()
            # self.multi_canvas[0].fill_circle(xpos, ypos, 10)
            # self.multi_canvas[0].flush()
            pos = self.debug_draw.screen_to_world((xpos, ypos))
            pos = pos.x, pos.y
            self.framework.on_mouse_move(pos)

        


        
        self.multi_canvas[1].on_mouse_down(on_mouse_down)
        self.multi_canvas[1].on_mouse_up(on_mouse_up)
        self.multi_canvas[1].on_mouse_move(on_mouse_move)
        


        # Register mouse click callback
        # self.multi_canvas[0].on_mouse_down(handle_mouse_down)
        # self.multi_canvas[1].on_mouse_down(handle_mouse_down)
        # self.canvas.on_mouse_down(handle_mouse_down)

        IPython.display.display(self.out)
        
        with self.out:
            IPython.display.display(self.multi_canvas)
        
    def run(self):

        
        target_fps = 30
        dt_desired_ms = (1.0/target_fps)*1000.0
        dt_desired_s = dt_desired_ms/1000.0
        
        print("dt_desired_ms", dt_desired_ms)
        print("dt_desired_s ", dt_desired_s)
        runtime = 100
        n = int(runtime / dt_desired_s)
        print(n)
        
        for i in range(n):
            
            t0 = time.time()
            
            canvas = self.multi_canvas[self.flip_bit]
            next_canvas = self.multi_canvas[not self.flip_bit]
            self.framework.step(dt_desired_s )
            with hold_canvas(next_canvas):
                self.debug_draw.canvas = next_canvas
                self.world.draw_debug_data()
            # break
            canvas.clear()
            
            t1 = time.time()
            d = t1 - t0
            if(d < dt_desired_s):
                time.sleep(dt_desired_s-d)


            self.flip_bit = not self.flip_bit

    

In [4]:
   

testbed = Testbed(guiType=IPyCanvasTestbedGui, gui_kwargs=dict())
testbed.setExample(Bridge)

stopped = Event()
def loop():
    testbed.run()
Thread(target=loop).start()  


scale 1.0


Output()

In [5]:
# stopped.set()

dt_desired_ms 33.333333333333336
dt_desired_s  0.03333333333333333
3000
